In [3]:
!pip install music21
!pip install --upgrade pip

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.8/22.8 MB 54.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.4/199.4 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 31.1 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 23.3.2
    Uninstalling pip-23.3.2:
      Successfully uninstalled pip-23.3.2


In [ ]:
# Normal
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from collections import Counter

# os
import os
import shutil
import pickle

# music21 
from music21 import *
from music21 import converter, instrument, note, chord

# tf imports
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization

In [ ]:
# music_gen_utils
def get_notes_and_chords(file):
    notes_chords_list = []
    for file_path in os.listdir(file):
        # Load MIDI file
        midi_stream = converter.parse(file+file_path)
        # Extract notes and chords from each part
        notes_and_chords = []
        for part in midi_stream.parts:
            for element in part.flat:
                if isinstance(element, note.Note):
                    notes_and_chords.append(element.nameWithOctave)
                elif isinstance(element, chord.Chord):
                    notes_and_chords.append(":".join([i.nameWithOctave for i in element.pitches]))
        notes_chords_list.append(notes_and_chords)         
    return notes_chords_list

def make_dataset(melodies_list, length=10):
    features_list = []
    labels_list = []
    for melody in melodies_list:
        melody_data = []
        pred_data = []
        for i in range(len(melody)-length-1):
            melody_data.append(melody[i:i+length])
            pred_data.append(melody[i+length])
        features_list.extend(melody_data)
        labels_list.extend(pred_data)
    return np.array(features_list), np.array(labels_list)

def ohe_labels(labels):
    label_elements = np.unique(labels)
    labels_enco = np.arange(0, len(label_elements))
    labels_mapping = dict(zip(label_elements, labels_enco))
    labels_reverse_mapping = dict(zip(labels_enco, label_elements))
    labels_ohe =  []
    for i in labels:
        label_array = np.zeros((len(label_elements), ))
        label_array[labels_mapping[i]] = 1
        labels_ohe.append(label_array)
    return np.array(labels_ohe), labels_mapping, labels_reverse_mapping 


*Loading Data and Parsing Data*

Getting Notes and Chords from the music data

In [6]:
# with open('beeth_notes_chords.pkl', 'wb') as file:
#     pickle.dump(notes_chords, file)

In [7]:
with open('/kaggle/input/beeth-notes-chords/beeth_notes_chords.pkl', 'rb') as file:
    notes_chords = pickle.load(file)

In [8]:
print("total number of melodies = ", len(notes_chords))

total number of melodies =  29


In [9]:
melodies_lengths = [len(i) for i in notes_chords]
melodies_lengths

[1586,
 4562,
 1369,
 313,
 2270,
 6379,
 5155,
 4761,
 876,
 5605,
 915,
 1757,
 1188,
 2669,
 5462,
 3589,
 3127,
 6908,
 4993,
 597,
 2454,
 1186,
 1036,
 662,
 2005,
 1150,
 2361,
 4261,
 2117]

Making a dataset of features and labels from above obatined notes and chords.

In [11]:
length = 30
features, labels = make_dataset(notes_chords, length)

In [12]:
features[0], labels[0]

(array(['D4', 'E-4', 'F4', 'A4:G4', 'F4', 'G4', 'A4', 'B-4', 'F4', 'A4:G4',
        'F4', 'G4', 'A4', 'B-4', 'A4', 'B-4', 'C5', 'D5', 'E5', 'B-4:E5',
        'A4:F5', 'F5:E-5', 'D5', 'E-5', 'F5', 'G5', 'C5', 'D5:C5', 'B4',
        'C5'], dtype='<U29'),
 'D5')

Getting all notes and chords in a single array

In [13]:
all_notes = np.concatenate((features.ravel(), labels))

Getting unique notes and chords

In [14]:
len(all_notes) # 890934
unique_notes = np.unique(all_notes)
len(unique_notes) # 3954
print("Number of unique notes or chords in the corpus: ", len(unique_notes))

Number of unique notes or chords in the corpus:  3954


In [19]:
sorted_notes = np.sort(unique_notes)
mapping_numbers = np.arange(1, len(unique_notes)+1, 1)

In [20]:
len(mapping_numbers) # 3954
norm_map_numbers = (mapping_numbers - min(mapping_numbers))/(max(mapping_numbers)-min(mapping_numbers))

In [21]:
norm_map_numbers = norm_map_numbers + 0.01

Mapping notes and chords to values from 0.01 and 1.01

In [22]:
mapping = dict(zip(sorted_notes, norm_map_numbers))
reverse_mapping = dict(zip(norm_map_numbers, sorted_notes))

Converting the notes to normalized numbers using the above establish mapping

In [23]:
features_norm = [mapping[note] for note in features.ravel()]
features_norm = np.array(features_norm).reshape((-1, length))

One hot encoding labels

In [154]:
# Finding number of unique elements in labels
len(np.unique(labels)) # 3916
len(labels) # 80994

80414

In [25]:
labels_encoded, labels_mapping, labels_reverse_mapping = ohe_labels(labels)

In [26]:
features_norm.shape, labels_encoded.shape

((80414, 30), (80414, 3916))

Splitting the dataset into training and seeding

In [27]:
from sklearn.model_selection import train_test_split
x_train, x_seed, y_train, y_seed = train_test_split(features_norm, labels_encoded, test_size=0.01, random_state=42)
# x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.08, random_state=42)

In [28]:
x_train = x_train.reshape((-1, length, 1))

In [ ]:
# y_train1 = y_train.reshape((*y_train.shape))
# y_valid1 = y_valid.reshape((*y_valid.shape))

In [31]:
len(x_seed)

805

In [32]:
len(x_train)

79609

Loading the model

In [39]:
model = Sequential()
model.add(LSTM(len(mapping_numbers)//4, input_shape=(x_train.shape[1], x_train.shape[2]), return_sequences=False))
model.add(Dense(512))
model.add(BatchNormalization())
model.add(Dense(448))
model.add(BatchNormalization())
model.add(Dense(y_train.shape[1], activation='softmax'))
# model.compile(loss='categorical_crossentropy', optimizer='Adam')

# callbacks = [EarlyStopping(
#                 monitor="loss",
#                 min_delta=0.002,
#                 patience=10,
#                 verbose=1,
#                 mode="auto",
#                 baseline=None,
#                 restore_best_weights=True,
#             ),
#                 ReduceLROnPlateau(
#                 monitor="loss",
#                 factor=0.1,
#                 patience=4,
#                 verbose=1,
#                 mode="auto",
#                 min_delta=0.01,
#                 cooldown=0,
#                 min_lr=1e-9
#             )]

/opt/conda/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [40]:
# saving model's architecture
model_json = model.to_json()
with open('beeth_30.json', 'w') as file:
    file.write(model_json)

# saving model's weights
model.load_weights('/kaggle/input/beeth-30/beeth_30.keras')

Training the model

Generating music using the seed split

In [97]:
np.save('beeth_seed.npy', x_seed)

In [144]:
class Generator:
    def __init__(self, model, x_seed, mapping={}, reverse_mapping={}, labels_reverse_mapping={}, name=""):
        self.model = model
        self.playlist = []
        self.counter = 0
        self.x_seed = x_seed
        self.length = x_seed.shape[-1]
        self.mapping = mapping
        self.reverse_mapping = reverse_mapping
        self.labels_reverse_mapping = labels_reverse_mapping
        self.name = name
        self.repeat_patience = 2
    
    def convert_to_midi(self, notes_list):
        melody_stream = []
        time_offset = 0
        for item in notes_list:
            if (":" in item or item.isdigit()):
                chord_notes = item.split(":")
                note_objects = [note.Note(n) for n in chord_notes]
                chord_obj = chord.Chord(note_objects)
                chord_obj.offset = time_offset
                melody_stream.append(chord_obj)
            else: 
                single_note = note.Note(item)
                single_note.offset = time_offset
                melody_stream.append(single_note)
            time_offset += 0.5
            
        melody_stream_midi = stream.Stream(melody_stream)
        return melody_stream_midi
    
    def generate_melody(self, note_count=200):
        seed_index = np.random.randint(0, len(x_seed) - 1)
        seed = self.x_seed[seed_index]
        generated_notes = []
        music = [self.reverse_mapping[i] for i in seed]
        music_norm = list(seed.ravel())
        prev_gen = seed
        i=0
        for _ in range(note_count):
            seed_reshaped = seed.reshape((1, length, 1))
            prediction = self.model.predict(seed_reshaped, verbose=0)
            index = np.argmax(prediction.ravel())
            music.append(self.labels_reverse_mapping[index])
            music_norm.append(self.mapping[self.labels_reverse_mapping[index]])
            seed = np.array(music_norm[-length:])
            if((i+1)%(self.length*self.repeat_patience)==0 and i>1):
                if(prev_gen == seed).all():
                    print("entered")
                    seed_index = np.random.randint(0, len(x_seed) - 1)
                    seed = x_seed[seed_index]
                prev_gen = seed
            i+=1
    
        melody = self.convert_to_midi(music)
        melody_midi = stream.Stream(melody)
        self.playlist.append((self.counter, melody_midi, self.length, note_count, self.name))                
        self.counter += 1;
        return melody_midi
    
    def generate_melody_rand(self, note_count=200):
        corpus = list(mapping.values())
        np.random.shuffle(corpus)
        seed = np.array(corpus[:self.length])
        generated_notes = []
        music = [self.reverse_mapping[i] for i in seed]
        music_norm = list(seed.ravel())
        generated_music = []
        prev_gen = seed
        i=0
        for _ in range(note_count):
            seed_reshaped = seed.reshape((1, length, 1))
            prediction = self.model.predict(seed_reshaped, verbose=0)
            index = np.argmax(prediction.ravel())
            music.append(self.labels_reverse_mapping[index])
            generated_music.append(music[-1])
            music_norm.append(self.mapping[self.labels_reverse_mapping[index]])
            seed = np.array(music_norm[-length:])
            if((i+1)%(self.length*self.repeat_patience)==0 and i>1):
                if(prev_gen == seed).all():
                    print("entered")
                    np.random.shuffle(corpus)
                    seed = np.array(corpus[:self.length])
                prev_gen = seed
            i+=1
    
        melody = self.convert_to_midi(generated_music)
        melody_midi = stream.Stream(melody)
        self.playlist.append((self.counter, melody_midi, self.length, note_count, self.name))                
        self.counter += 1;
        return melody_midi
    
    def save_melodies(self, direc_name = 'myPlaylist'):
        try:
            os.mkdir(direc_name)
        except FileExistsError:
            pass
        if(self.playlist==[]):
            print("The playlist is empty!")
            return
        for counter, melody, length, note_count, name in self.playlist[:self.counter]:
            melody.write('midi',f'{direc_name}/{counter}_{name}_generated_{length}_{note_count}.midi')
    
    def create_playlist(self, num_songs=2, note_count=200, direc_name=f"myMusic", zip_file=False, rand=False):
        melodies = []
        if not rand:
            for i in range(num_songs):
                melodies.append((i, self.generate_melody(note_count)))
        else:
            for i in range(num_songs):
                melodies.append((i, self.generate_melody_rand(note_count)))
        try:
            os.mkdir(direc_name)
        except:
            pass
        for i, melody in melodies:
            melody.write('midi',f'{direc_name}/{i}_{self.name}_generated_{self.length}_{note_count}.midi')
        if zip:
            zip_dir(direc_name, f"{direc_name}Zip")
        

In [145]:
generator = Generator(model, x_seed, mapping=mapping, reverse_mapping=reverse_mapping, labels_reverse_mapping=labels_reverse_mapping, name="Beeth")

In [141]:
melody = generator.generate_melody_rand(500)

In [142]:
generator.save_melodies()

In [149]:
count = 0

In [150]:
note_count = 200
num_songs = 2
generator.create_playlist(num_songs, note_count, direc_name = f"music{count}", zip_file=True)
count+=1

In [151]:
note_count = 200
num_songs = 2
generator.create_playlist(num_songs, note_count, direc_name = f"music{count}", zip_file=True, rand=False)
count+=1

Saving the music genearted

Saving the model

In [128]:
def del_files(dir_path):
    if os.path.isdir(dir_path):   
        dir_list = os.listdir(dir_path)
    else:
        os.remove(dir_path)
    for file in dir_list:
        file_path = os.path.join(dir_path, file)
        if os.path.isdir(file_path):
            shutil.rmtree(file)
        else:
            os.remove(file)

def zip_dir(from_file_path, to_file_path):
    shutil.make_archive(to_file_path, 'zip', from_file_path)


In [ ]:
del_files('/kaggle/working/')

In [136]:
zip_dir('/kaggle/working/myMusic', '/kaggle/working/myMusicZip')

In [137]:
x = np.random.randint(1, 100, (20, 30))

In [138]:
x[0].shape

(30,)